# Practicing Coordinate Transforms

In this notebook we will be practicing how to transform vectors between the various reference frames used for navigating and controlling an autonomous quadrotor. For this work we will be relying heavily on the [tf/transformations.py](https://github.com/ros/geometry/blob/hydro-devel/tf/src/tf/transformations.py) library.

In [2]:
from __future__ import division, print_function
import numpy as np
import tf.transformations as tft

# Variable Notation:
# v__x: vector expressed in "x" frame
# q_x_y: quaternion of "x" frame with respect to "y" frame
# p_x_y__z: position of "x" frame with respect to "y" frame expressed in "z" coordinates
# v_x_y__z: velocity of "x" frame with respect to "y" frame expressed in "z" coordinates
# R_x2y: rotation matrix that maps vector represented in frame "x" to representation in frame "y" (right-multiply column vec)
#
# Frame Subscripts:
# m = marker frame (x-right, y-up, z-out when looking at marker)
# dc = downward-facing camera
# fc = forward-facing camera
# bu = body-up frame (x-forward, y-left, z-up, similar to ENU)
# bd = body-down frame (x-forward, y-right, z-down, similar to NED)
# lenu = local East-North-Up world frame ("local" implies that it may not be aligned with east and north, but z is up)
# lned = local North-East-Down world frame ("local" implies that it may not be aligned with north and east, but z is down)

In [1]:
# Concept Questions: Just based on your understanding of what the different reference frames represent,
# can you find the following velocities just by inspection?:
v_bd_bd__bd = [0., 0., 0.]
v_bu_bu__bu = [0., 0., 0.]
v_fc_fc__fc = [0., 0., 0.]
v_dc_dc__dc = [0., 0., 0.]
v_bd_bd__fc = [0., 0., 0.]
v_dc_dc__lned = [0., 0., 0.]
v_bd_fc__lned = [0., 0., 0.]
v_dc_bu__lenu = [0., 0., 0.]

## Fixed/Static Relative Rotations

This rotation matrices are constant, they don't change regardless of the motion of the quadrotor

In [3]:
class StaticTransforms():
    # local ENU and local NED
    R_lenu2lned = tft.rotation_matrix(np.pi, (1,0,0))

    # body-up and body-down
    R_bu2bd = tft.rotation_matrix(np.pi, (1,0,0))

    # downward camera and body-down
    R_dc2bd = tft.identity_matrix()

    # forward camera and body-down
    R_fc2bd = np.array([[0.0, 0.0, 1.0, 0.0],
                        [1.0, 0.0, 0.0, 0.0],
                        [0.0, 1.0, 0.0, 0.0],
                        [0.0, 0.0, 0.0, 1.0]])
    
    # Find inverse rotation matrices 
    R_lned2lenu = R_lenu2lned.T
    R_bd2bu = R_bu2bd.T
    R_bd2dc = R_dc2bd.T
    R_bd2fc = R_fc2bd.T
    
    # Find concatenated rotation matrices from downward-camera to forward-camera
    R_dc2fc = tft.concatenate_matrices(R_bd2fc, R_dc2bd)
    R_fc2dc = R_dc2fc.T
    R_dc2bu = tft.concatenate_matrices(R_bd2bu, R_dc2bd)
    R_bu2dc = R_dc2bu.T
    R_fc2bu = tft.concatenate_matrices(R_bd2bu, R_fc2bd)
    R_bu2fc = R_fc2bu.T
    
    def __init__(self):
        pass
    
    def coord_transform(self, v__fin, fin, fout):
        ''' transform vector v which is represented in frame fin into its representation in frame fout
        Args:
        - v__fin: 3D vector represented in fin coordinates
        - fin: string describing input coordinate frame (bd, bu, fc, dc, lned, lenu)
        - fout: string describing output coordinate frame (bd, bu, fc, dc, lned, lenu)
        Returns
        - v__fout: vector v represent in fout coordinates
        '''
        
        # trivial transform, checking input shape
        if fin==fout:
            v4__fin = list(v__fin)+[0.0]
            R = tft.identity_matrix()
            v4__fout = np.dot(R, v4__fin)
            v__fout = v4__fout[0:3]
            return v__fout
        
        # check for existence of rotation matrix
        R_str = 'R_{}2{}'.format(fin, fout)
        try:
            R_i2o = getattr(self, R_str)
        except AttributeError:
            err = 'No static transform exists from {} to {}.'.format(fin, fout)
            err += ' Are you sure these frames are not moving relative to each other?'
            raise AttributeError(err)
        
        # perform transform
        v4__fin = list(v__fin) + [0.0]
        v4__fout = np.dot(R_i2o, v4__fin)
        v__fout = v4__fout[0:3]
        return v__fout
 
st = StaticTransforms()

In [8]:
# Let's assume the quadrotor has some velocity v1_bd_lned__bd which is the velocity of the quadrotor 
# body-down frame with respect to the local NED world frame expressed in the body-down frame.
# Using the fixed relative rotations, calculate it's expression in the body-up, downward-camera, and forward-camera frames
v1_bd_lned__bd = [1.0, 0.0, 0.0]
v1_bd_lned__bu = st.coord_transform(v1_bd_lned__bd, "bd","bu")
v1_bd_lned__dc = st.coord_transform(v1_bd_lned__bd, "bd","dc")
v1_bd_lned__fc = st.coord_transform(v1_bd_lned__bd, "bd","fc")
print(v1_bd_lned__bu)
print(v1_bd_lned__dc)
print(v1_bd_lned__fc)


[ 1.  0.  0.]
[ 1.  0.  0.]
[ 0.  0.  1.]


In [ ]:
# Let's assume the quadrotor has some velocity v2_bd_lned__bd which is the velocity of the quadrotor 
# body-down frame with respect to the local NED world frame expressed in the body-down frame.
# Using the fixed relative rotations, calculate its expression in the body-up, downward-camera, and forward-camera frames
v2_bd_lned__bd = [0.147, 0.798, 1.221]
v2_bd_lned__bu = st.coord_transform(v2_bd_lned__bd, "bd","bu")
v2_bd_lned__dc = st.coord_transform(v2_bd_lned__bd, "bd","dc")
v2_bd_lned__fc = st.coord_transform(v2_bd_lned__bd, "bd","fc")
print(v2_bd_lned__bu)
print(v2_bd_lned__dc)
print(v2_bd_lned__fc)


In [10]:
# Let's assume the quadrotor has some velocity v3_dc_lenu__dc which is the velocity of the quadrotor 
# downward-camera frame with respect to the local ENU world frame expressed in the downward-camera frame.
# Using the static transforms, calculate it's expression in the body-down, forward-camera, and body-up frames
v3_dc_lenu__dc = [4.853, 2.979, 1.884]
v3_dc_lenu__bd = st.coord_transform(v2_bd_lned__bd, "dc","bd")
v3_dc_lenu__fc = st.coord_transform(v2_bd_lned__bd, "dc","fc")
v3_dc_lenu__bu = st.coord_transform(v2_bd_lned__bd, "dc","bu")
print(v3_dc_lenu__bd)
print(v3_dc_lenu__fc)
print(v3_dc_lenu__bu)

[ 0.147  0.798  1.221]
[ 0.798  1.221  0.147]
[ 0.147 -0.798 -1.221]


In [11]:
# Let's assume the quadrotor has some velocity v4_fc_lenu__bd which is the velocity of the quadrotor 
# forward-camera frame with respect to the local ENU world frame expressed in the body-down frame.
# Using the static transforms, calculate it's expression in the forward-camera, downward-camera and and body-up frames
v4_fc_lenu__bd = [0.0, 0.0, -1.0]
v4_fc_lenu__fc = st.coord_transform(v2_bd_lned__bd, "bd","fc")
v4_fc_lenu__dc = st.coord_transform(v2_bd_lned__bd, "bd","dc")
v4_fc_lenu__bu = st.coord_transform(v2_bd_lned__bd, "bd","bu")
print(v4_fc_lenu__fc)
print(v4_fc_lenu__dc)
print(v4_fc_lenu__bu)

[ 0.798  1.221  0.147]
[ 0.147  0.798  1.221]
[ 0.147 -0.798 -1.221]


## Dynamic Relative Rotations

In the previous section we looked at reference frames that remain fixed relative to one another (i.e. reference frames that are all attached to quadrotor or reference frames, or reference frames that are all associated with inertial local world frames). Now were going to look at reference frames that may be moving relative to one another, such as a body-fixed frame and the local world frame.

For such moving frames, we often can't create rotation matrices by inspection. Furthermore, such rotations need to be calculated automatically by the quadrotor's flight computer in real-time. This is the job of the _state estimator_ that runs onboard the flight computer. The state estimator will output estimates of the relative rotations between local world frame and the body frame.

More specifically, the topic `mavros/local_position/pose` provides [`PoseStamped`](http://docs.ros.org/melodic/api/geometry_msgs/html/msg/PoseStamped.html) messages that contain the orientation of the body-down frame with respect to the local ENU frame in the form of a [`Quaternion`](http://docs.ros.org/melodic/api/geometry_msgs/html/msg/Quaternion.html).

Therefore, when using MAVROS, you could use a assignment such as the one below to find `q_bu_lenu`:

```
q_bu_lenu = pose_stamped_msg.pose.orientation
```

Below is a function that we can use when flying the drone to transforms vectors in an arbitrary reference frame to the local ENU reference frame, assuming that we have access to the `mavros/local_position/pose` topic to tell us `q_bu_lenu` (in this case we assume they are velocity vectors) 

In [ ]:
def get_lenu_velocity(q_bu_lenu, v__fin, fin='bu', static_transforms=None):
        '''tranforms a vector represented in fin frame to vector in lenu frame
        Args:
        - v__fin: 3D vector represented in input frame coordinates
        - fin: string describing input coordinate frame (bd, bu, fc, dc)
        Returns:
        - v__lenu: 3D vector v represented in local ENU world frame
        '''

        # create static transforms if none given
        if static_transforms is None:
            static_transforms = StaticTransforms()

        if fin=='lenu':
            v__lenu = v__fin

        elif fin=='lned':
            v__lenu = static_transforms.coord_transform(v__fin, 'lned', 'lenu')

        else:
            # create rotation matrix from quaternion
            R_bu2lenu = tft.quaternion_matrix(q_bu_lenu)
            
            # represent vector v in body-down coordinates
            v__bu = static_transforms.coord_transform(v__fin, fin, 'bu')
            
            # calculate lenu representation of v
            v__lenu = np.dot(R_bu2lenu, list(v__bu)+[0.0])

        v__lenu = np.array(v__lenu[0:3])
        return v__lenu

In [ ]:
tft.quaternion_from_matrix(tft.identity_matrix())

## ROS tf2 Library

The problems we have attempted to address in this module (i.e. managing multiple reference frames) are by no means unique to quadrotors and we are not the first people to write functions to solve such problems. The functionality of managing different reference frames is ubiquitous throughout robotics, aerospace engineering, mechanical engineering, computer graphics, etc. and many libraries have been written for handling such functionality. When working with ROS, the most important of such libraries is the [tf (now tf2) library](http://wiki.ros.org/tf2). While we have access to this library on the drone, we have not made use of it here because it obscures some of the underlying mathematics that we hope for you to learn and it requires additional setup steps when defining new frames that we don't intend to teach. If you are curious to know more about how ROS manages large numbers of reference frames simultaneously, we encourage you to read up more on `tf`.

__NOTE:__ `tf` in the context of ROS should not be confused with TensorFlow which is often abbreviated as tf in code. These libraries have completely different purposes.